In [ ]:
!pip install gpytorch
!pip install ConfigSpace
!pip install openml

In [ ]:
!pip install tabpfn

In [ ]:
!pip install scikit-learn==1.1.3

In [ ]:
import torch
print(torch.version.cuda)  # 설치된 PyTorch가 지원하는 CUDA 버전
print(torch.cuda.is_available())  # GPU 사용 가능 여부

In [ ]:
import openml
# OpenML-CC18 벤치마크 데이터셋 로드
benchmark_suite = openml.study.get_suite('OpenML-CC18')

# 데이터셋 목록 출력
print(benchmark_suite.data)

In [ ]:
# OpenML API Key 설정 (웹사이트에서 발급받은 API 키로 교체)
openml.config.apikey = '046eaf7fbf653b9d44127e029f473fb7'

In [ ]:
data_list = [11,14,15,16,18,22,23,29,31,37,50,54,188,1049,1050,1063,1068,1462,1464,1480,1494,1510,6332,23381,40966,40975,40982,40994]

In [ ]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tabpfn import TabPFNClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss



# 결과를 저장할 딕셔너리
datasets = {}
acc = []
roc = []
ce = []

for dataset_id in data_list:


    # 데이터셋 로드
    dataset = openml.datasets.get_dataset(dataset_id)

    # 데이터셋 이름 가져오기 (고유 이름으로 사용)
    dataset_name = dataset.name.replace(" ", "_")  # 공백 제거
    # print(f"Loading dataset: {dataset_name} (ID: {dataset_id})")

    # 데이터셋을 DataFrame으로 가져오기
    X, y, _, attribute_names = dataset.get_data(target=dataset.default_target_attribute)

    # print(f'Dataset shape : {X.shape}')
    # print(f"Dataset dtypes:\n{X.dtypes}")
   
    # 비수치형 데이터 식별 및 변환
    for col in X.columns:
        if X[col].dtype.name == 'category':
            # print(f"Converting categorical column: {col}")
            X[col] = X[col].cat.codes  # category를 정수형 코드로 변환
        elif not np.issubdtype(X[col].dtype, np.number):
            # print(f"Converting non-numeric column: {col}")
            X[col] = pd.to_numeric(X[col], errors='coerce')  # 문자열 데이터는 수치형으로 변환

    # 결측값 처리
    X = X.fillna(0)  # 결측값을 0으로 대체

    # 데이터프레임을 numpy 배열로 변환
    X = X.to_numpy()
    y = y.to_numpy()

    # 데이터 타입 확인
    # print(f"X dtype: {X.dtype}, y dtype: {y.dtype}")

# 데이터 크기 제한 및 train/test 분리
    if len(X) > 1024:
        # 테스트 데이터 분리
        X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
        
        # 학습 데이터 크기가 1024보다 작은 경우 처리
        train_size = min(1024, len(X_train_full)-1)
        X_train, _, y_train, _ = train_test_split(X_train_full, y_train_full, train_size=train_size, random_state=42)
    else:
        # 데이터 크기가 작다면 그대로 분리
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # print(f"X_train shape: {X_train.shape}")
    # print(f"X_test shape: {X_test.shape}")

    # TabPFN 모델 생성
    classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

    # 모델 학습
    classifier.fit(X_train, y_train)

    y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

    # 데이터 확인
    # print(f"Dataset '{dataset_name}' loaded with shape: {df.shape}")
    accu = accuracy_score(y_test, y_eval)
    acc.append(accu)
    print(f'Loading dataset: {dataset_name} (ID: {dataset_id}) Accuracy : {accu}')


    # 예측 (확률 반환)
    y_prob = classifier.predict_proba(X_test)
    

    # y_test가 2D 배열인지 확인하고 변환
    if len(y_test.shape) > 1:
        y_test = y_test.ravel()

  
    # ROC AUC OVO 계산 (이진 분류의 경우)
    if y_prob.ndim == 2 and y_prob.shape[1] == 2:
        roc_auc = roc_auc_score(y_test, y_prob[:, 1])
    else:
        # 다중 클래스의 경우 OVO
        roc_auc = roc_auc_score(y_test, y_prob, multi_class='ovo', average='macro')


    roc.append(roc_auc)

    print(f"Loading dataset: {dataset_name} (ID: {dataset_id}) ROC AUC OVO Score: {roc_auc:.4f}")


print(f'Mean Acc. : {np.mean(acc)}, Mean AUC OVO : {np.mean(roc)}')